<a href="https://colab.research.google.com/github/verma7/AdventOfCode2021/blob/master/RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# RL

In [3]:
!pip install tf-agents[reverb]
!pip install tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 38.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 110.3 MB/s eta 0:00:00
  

In [4]:
import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [5]:
!pip install galois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 49.5 MB/s eta 0:00:00


In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np
import random
import galois
import pandas as pd

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

from tf_agents.policies import random_py_policy
from tf_agents.policies import random_tf_policy
from tf_agents.metrics import py_metrics
from tf_agents.metrics import tf_metrics
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.drivers import dynamic_step_driver

from tf_agents import specs
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.replay_buffers import py_uniform_replay_buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

from tf_agents.train.utils import strategy_utils


In [71]:
def get_num_ones(w):
    "Returns a vector containing the number of ones in the bit representation in each element of the GF(2^w) field."
    N = 2 ** w
    GF = galois.GF(N)
    num_ones = np.zeros(N)
    for i in range(N):
        for j in range(w):
            v = np.multiply(GF(i), GF(2 ** j))
            num_ones[i] += bin(v).count("1")
    return num_ones

def count_ones(GC, n, m, ones):
    sum_ones = 0
    for i in range(n):
        for j in range(m):
            sum_ones += ones[int(GC[i][j])]
    return sum_ones

def swap(x, y, state_x, state_y, state):
  xi = 0
  for i in range(len(state)):
    if state[i] == state_x:
      if xi == x:
        break
      xi += 1

  xj = 0
  for j in range(len(state)):
    if state[j] == state_y:
      if xj == y:
        break
      xj += 1

  tmp = state[i]
  state[i] = state[j]
  state[j] = tmp

def normalize_bitmatrix(GF, GC, m, ONES):
  # Make the first row 1s by dividing all the columns by its first element.
  M = np.zeros((m, m))
  for i in range(m):
      for j in range(m):
          if GC[0][j] == 0:
              continue
          M[i][j] = np.divide(GF(int(GC[i][j])), GF(int(GC[0][j])))
#    print(M)

  # For each row, pick one of the elements that minimizes the number of ones and divide the row by it.
  for i in range(1, m):
      min_ones = 1e8
      best_C = None

      for j in range(m):
          if M[i][j] == 0:
              continue

          C = np.zeros(m)
          sum_ones = 0
          C_ones = []
          for k in range(m):
              C[k] = np.divide(GF(int(M[i][k])), GF(int(M[i][j])))
              sum_ones += ONES[int(C[k])]
              C_ones.append(ONES[int(C[k])])
          if sum_ones < min_ones:
              min_ones = sum_ones
              best_C = C
      for j in range(m):
          if best_C is not None:
              M[i][j] = best_C[j]
  return M

def good_cauchy_matrix(GF, m, N, state):
  GC = np.zeros((m, m))
  x = 0
  y = 0
  for i in range(N):
      if state[i] == 0:
        y = 0
        for j in range(N):
            if state[j] == 1:
              sum = GF(i) + GF(j)
              if sum == 0:
                  continue
              GC[x][y] = sum ** -1
              y += 1
        x += 1
  return GC

def simulate_action(action, m, N, state, GF, ONES):
  max_action = m * m + 2 * m * N - 1
  if action < m * m:
    x = action // m
    y = action % m
    swap(x, y, 0, 1, state)
  elif action < m * m + m * N:
    denom = m * N
    offset = m * m
    x = (action - offset) // denom
    y = (action - offset) % denom
    swap(x, y, 0, 2, state)
  elif action < m * m + 2 * m * N:
    denom = m * N
    offset = m * m + m * N
    x = (action - offset) // denom
    y = (action - offset) % denom
    swap(x, y, 1, 2, state)
  else:
    raise ValueError(f'`action` should be between [0, {max_action}), but is {action}')

  GC = good_cauchy_matrix(GF, m, N, state)
  NGC = normalize_bitmatrix(GF, GC, m, ONES)
  return count_ones(NGC, m, m, ONES)

def random_weighted_action(m, N, state, num_ones):
  """Returns an action"""
  weights = np.zeros(m * m + 2 * m * N)
  GF = galois.GF(N)
  ONES = get_num_ones(N)

  best_action = 0
  best_reward = 0
  for action in range(len(weights)):
    state_copy = state.copy()
    weights[action] = num_ones - simulate_action(action, m, N, state_copy, GF, ONES)
    print(f'action = {action}, reward = {weights[action]}, prev num_ones = {num_ones}')
    if weights[action] > best_reward:
      best_reward = weights[action]
      best_action = action
  return best_action

In [73]:
class CauchyEnv(py_environment.PyEnvironment):

  def __init__(self, W, m, vec, num_ones_list):
    self._W = W
    self._N = 2 ** self._W
    self._m = m
    self._GF = galois.GF(self._N)
    self._ONES = get_num_ones(self._W)
    self._num_ones_list = num_ones_list

    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int64, minimum=0, maximum=self._m * self._m + 2 * self._m * self._N - 1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(self._N, ), dtype=np.int32, minimum=0, maximum=2, name='observation')
    self._initialize(vec)

  def _initialize(self, vec):
    self._best_num_ones = 1e8
    self._vec = vec
    self._state = [0] * self._N
    for i in self._vec[:self._m]:
        self._state[i] = 0
    for i in self._vec[self._m:2*self._m]:
        self._state[i] = 1
    for i in self._vec[2*self._m:]:
        self._state[i] = 2
    self._GC = good_cauchy_matrix(self._GF, self._m, self._N, self._state)
    NGC = normalize_bitmatrix(self._GF, self._GC, self._m, self._ONES)
    self._num_ones = count_ones(NGC, self._m, self._m, self._ONES)
    self._best_num_ones = self._num_ones
    self._time_steps_without_improvement = 0

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._initialize(self._vec)
    return ts.restart(np.array(self._state, dtype=np.int32))

  def _step(self, action):
    new_num_ones = simulate_action(action, self._m, self._N, self._state, self._GF, self._ONES)
    reward = self._num_ones - new_num_ones
    self._num_ones = new_num_ones
    self._num_ones_list.append(new_num_ones)
    if new_num_ones < self._best_num_ones:
      self._best_num_ones = new_num_ones
      # print(f'Best num ones = {new_num_ones}')
      self._time_steps_without_improvement = 0
    else:
      self._time_steps_without_improvement += 1
    # print(f'Time steps without improvement: {self._time_steps_without_improvement}, best num ones = {self._best_num_ones}')
    if self._time_steps_without_improvement == 10:
      self._time_steps_without_improvement = 0
      self._initialize(self._vec)
      return ts.termination(np.array(self._state, dtype=np.int32), reward)
    else:
      return ts.transition(np.array(self._state, dtype=np.int32), reward=reward, discount=1.0)

In [74]:
w = 4
m = 4
N = 2 ** w
vec = [ i for i in range(N)]
random.shuffle(vec)


max_episodes = 10
num_episodes = 0

num_ones = []
env = CauchyEnv(w, m, vec, num_ones)
tf_env = tf_py_environment.TFPyEnvironment(env)

while (num_episodes < max_episodes):
  time_step = tf_env.reset()
  num_episodes += 1
  num_ones.clear()

  while not time_step.is_last():
    action = random_weighted_action(env._m, env._N, env._state, env._num_ones)
    print(f"action = {action}")
    time_step = tf_env.step(action)

  if num_ones:
    print(num_ones)
    print(f'Min num_ones: {np.min(num_ones)}')

action = 0, reward = -281.0, prev num_ones = 101.0
action = 1, reward = -323.0, prev num_ones = 101.0
action = 2, reward = -308.0, prev num_ones = 101.0
action = 3, reward = -373.0, prev num_ones = 101.0
action = 4, reward = -301.0, prev num_ones = 101.0
action = 5, reward = -334.0, prev num_ones = 101.0
action = 6, reward = -372.0, prev num_ones = 101.0
action = 7, reward = -307.0, prev num_ones = 101.0
action = 8, reward = -306.0, prev num_ones = 101.0
action = 9, reward = -327.0, prev num_ones = 101.0
action = 10, reward = -357.0, prev num_ones = 101.0
action = 11, reward = -311.0, prev num_ones = 101.0
action = 12, reward = -294.0, prev num_ones = 101.0
action = 13, reward = -317.0, prev num_ones = 101.0
action = 14, reward = -299.0, prev num_ones = 101.0
action = 15, reward = -311.0, prev num_ones = 101.0
action = 16, reward = -299.0, prev num_ones = 101.0
action = 17, reward = -295.0, prev num_ones = 101.0
action = 18, reward = -344.0, prev num_ones = 101.0
action = 19, reward = 

KeyboardInterrupt: 

In [13]:
w = 4
m = 4
N = 2 ** w
vec = [ i for i in range(N)]
random.shuffle(vec)

R = np.zeros((N, N))
num_ones = []
num_actions = m * m + 2 * m * N
for i in range(num_actions):
  environment = CauchyEnv(w, m, vec, num_ones)
  time_step = environment.reset()
  time_step = environment.step(i)
  R[i//N][i%N] = time_step.reward

print(num_ones)
for i in range(N):
  print(' '.join([f"{R[i][j]:3.0f}" for j in range(N)]))

[103.0, 101.0, 100.0, 98.0, 99.0, 92.0, 107.0, 101.0, 94.0, 96.0, 100.0, 97.0, 99.0, 100.0, 98.0, 106.0, 101.0, 98.0, 97.0, 103.0, 102.0, 99.0, 104.0, 103.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 100.0, 105.0, 89.0, 97.0, 98.0, 100.0, 96.0, 97.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0, 99.0]
  2   0  -1  -3  -2  -9   6   0  -7  -5  -1  -4  -2  -1  -3   5

In [17]:
W = 4
N = 5
vec = [ i for i in range(2 ** W)]
random.shuffle(vec)
env = CauchyEnv(W, N, vec, num_ones)
tf_env = tf_py_environment.TFPyEnvironment(env)

tf_policy = random_tf_policy.RandomTFPolicy(action_spec=tf_env.action_spec(),
                                            time_step_spec=tf_env.time_step_spec())


num_episodes = tf_metrics.NumberOfEpisodes()
env_steps = tf_metrics.EnvironmentSteps()
max_return = tf_metrics.MaxReturnMetric()
replay_buffer = []
observers = [num_episodes, env_steps, max_return, replay_buffer.append]
driver = dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env, tf_policy, observers, num_episodes=100)

# Initial driver.run will reset the environment and initialize the policy.
final_time_step, policy_state = driver.run()

print('final_time_step', final_time_step)
print('Number of Steps: ', env_steps.result().numpy())
print('Number of Episodes: ', num_episodes.result().numpy())
print('Max Return: ', max_return.result().numpy())
print('Min num_ones: ', np.min(num_ones))


for epsilon in np.arange(0, 1.1, 0.1):
  num_ones = []
  environment = CauchyEnv(W, N, vec, num_ones)
  tf_env = tf_py_environment.TFPyEnvironment(environment)

  q_net = q_network.QNetwork(
      tf_env.time_step_spec().observation,
      tf_env.action_spec(),
      fc_layer_params=(100,))

  agent = dqn_agent.DqnAgent(
      tf_env.time_step_spec(),
      tf_env.action_spec(),
      q_network=q_net,
      epsilon_greedy=epsilon,
      optimizer=tf.compat.v1.train.AdamOptimizer(1e-4))

  collect_steps_per_iteration = 100
  avg_return_metric = tf_metrics.AverageReturnMetric()
  avg_episode_length_metric = tf_metrics.AverageEpisodeLengthMetric()
  observers = [] #avg_return_metric, avg_episode_length_metric]
  collect_op = dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env,
    agent.collect_policy,
    observers=observers,
    num_episodes=collect_steps_per_iteration).run()

  print(num_ones)
  print(f"Best num_ones with epsilon greedy={epsilon} is {np.min(num_ones)}")
  # print(f"Avg return = {avg_return_metric.result().numpy()}")
  # print(f"Avg episode length = {avg_episode_length_metric.result().numpy()}")


final_time_step TimeStep(
{'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-5.], dtype=float32)>,
 'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 16), dtype=int32, numpy=array([[1, 0, 2, 2, 1, 1, 2, 0, 1, 1, 2, 0, 2, 2, 0, 0]], dtype=int32)>})
Number of Steps:  1500
Number of Episodes:  100
Max Return:  -254.0
Min num_ones:  147.0
[160.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 160.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 160.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 160.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 160.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 160.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 155.0, 15

In [ ]:
learning_rate = 1e-5

W = 8
N = 10
vec = [ i for i in range(2 ** W)]
random.shuffle(vec)
num_ones = []
environment = CauchyEnv(W, N, vec, num_ones)
tf_env = tf_py_environment.TFPyEnvironment(environment)

eval_num_ones = []
eval_environment = CauchyEnv(W, N, vec, eval_num_ones)
tf_eval_env = tf_py_environment.TFPyEnvironment(eval_environment)

fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(tf_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate))

replay_buffer_capacity = 10000

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=replay_buffer_capacity)
replay_buffer.clear()

# Add an observer that adds to the replay buffer:
replay_observer = [replay_buffer.add_batch]

collect_steps_per_iteration = 1000
collect_op = dynamic_episode_driver.DynamicEpisodeDriver(
  tf_env,
  agent.collect_policy,
  observers=replay_observer,
  num_episodes=collect_steps_per_iteration).run()

print(num_ones)
print(f"Best num_ones = {np.min(num_ones)}")

In [ ]:
print(agent.collect_policy._epsilon)

In [ ]:
def compute_min_ones(environment, policy, num_ones, num_episodes=10):
  total_num_ones = 0
  actions = []
  for _ in range(num_episodes):
    time_step = environment.reset()
    num_ones.clear()
    actions.clear()
    rewards_list = []
    rewards = 0
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      rewards_list = np.concatenate((rewards_list, time_step.reward.numpy()))
      rewards += time_step.reward
      actions.append(action_step.action.numpy())
    print(actions)
    print(num_ones)
    print(rewards.numpy())
    print(rewards_list)
    total_num_ones += np.min(num_ones)
  return total_num_ones / num_episodes

In [ ]:
log_interval = 100
eval_interval = 100

num_train_steps = 100
df = pd.DataFrame()

dataset = replay_buffer.as_dataset(
    sample_batch_size=10,
    num_steps=2)
iterator = iter(dataset)

for step in range(num_train_steps):
  trajectories, _ = next(iterator)
  train_loss = agent.train(experience=trajectories).loss

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))
  if step % eval_interval == 0:
    min_ones = compute_min_ones(tf_eval_env, agent.collect_policy, eval_num_ones)
    print('step = {0}: min_ones = {1}'.format(step, min_ones))
  df = pd.concat([df, pd.DataFrame({'step': [step], 'min_ones': [min_ones], 'loss': [train_loss.numpy()]})])
print(df)

In [ ]:
print(agent.policy)

In [ ]:
df.set_index('step', inplace=True)
df['min_ones'].plot()